<a href="https://colab.research.google.com/github/andrewbeeksma/Maximizing-App-Profitability/blob/master/Maximizing_App_Popularity_for_Harvestly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maximizing App Profitability for Harvestly
In this project, I focus on profiling apps into different categories in order to distinguish which apps tend to be the most popular, and subsequently, the most profitable. Using this analysis, I hope to draw accurate conclusions that will inform Harvestly's software engineering practices and maximize our 'chances' at building a successful mobile application. Ensuring that our tech team deeply understands the data on the App Store and on Google Play will enable us to make practical, results-oriented decisions in every stage of the development process.

In [ ]:
#Import data from the App Store
from csv import reader
opened_file1 = open('AppleStore.csv')
read_file1 = reader(opened_file1)
app_store_data = list(read_file1)

#Import data from Google Play
opened_file2 = open('googleplaystore.csv')
read_file2 = reader(opened_file2)
google_play_data = list(read_file2)

#Create a function that explores a given dataset, which is expected to be a list of lists
#Take the starting and ending indices as arguments, with the option to print rows and columns
def explore_data(dataset, start, end, rows_and_columns=False):
    #create a subset of the dataset as specified by the arguments
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    #If rows_and_columns is true, then print the number of rows and columns in the dataset
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

explore_data(app_store_data, 0, 1, True)
print('\n\n')
explore_data(google_play_data, 0, 1, True)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Number of rows: 7198
Number of columns: 17



['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows: 10842
Number of columns: 13


In the code above, I am exploring what sort of data is in the *comma separated values* files, or CSV files. I am currently only exploring the header rows for each file to understand what each column represents and determine which columns will provide meaningful and relevant data for my intended analysis. If you would like to better understand these datasets, you can find information on the App Store [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) and information on Google Play [here](https://www.kaggle.com/lava18/google-play-store-apps). Note that before beginning my analysis, I must ensure that the data I analyze is accurate and relevant, which means I must first detect inaccurate and duplicate data. Furthermore, in order to apply the results of my analysis to Harvestly's goals, it would be best for me to remove all non-English apps and all apps that aren't free. This process is called *data cleaning*, and it is often said that data analysts spend roughly 80% of their time cleaning data, and the remaining 20% is spent analyzing this data. We will find this to be true for this project.

First, I will clean the data on the Google Play store. In the discussion section where I found the dataset, someone noted that row 10473 was missing a 'category' section, and all of the remaining columns were shifted over. The code below verifies that this row is missing data and ultimately deletes the row from the dataset.

In [ ]:
explore_data(google_play_data, 10473, 10474)
del google_play_data[10473] #delete row with missing data

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']




Next, it has come to my attention that there are a number of duplicate rows in the data. The code below verifies this fact by looping over the dataset and printing out the information for each application named 'Instagram'.

In [ ]:
for apps in google_play_data[1:]:
    name = apps[0]
    if name == 'Instagram':
        print(apps)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Note that the only column that varies in each row is the 4th column, which counts the total number of reviews. As I delete duplicate entries, then, I will only keep the row with the highest number of reviews. This will ensure that the data I am analyzing is the most recent. Now I will find out how many duplicate entries exist in the dataset.

In [ ]:
duplicate_apps = []
unique_apps = []

for app in google_play_data[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate entries:', len(duplicate_apps))

Number of duplicate entries: 1181


In order to remove all duplicate entries in my dataset, I will create a dictionary where each unique key is the application name and each value is the highest number of reviews for that application. Using this information, I can then create a new dataset with no duplicate entries.

In [ ]:
reviews_max = {}
for row in google_play_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
print('Number of unique entries:', len(reviews_max))

Number of unique entries: 9659


Now that I have a dictionary that assigns each application name to its maximum number of reviews, I can use this to clean my google_play_data set of duplicate entries. I will start by initializing a new list named google_play_clean, and an already_added list to keep myself from appending duplicate rows. So, with each iteration over the google_play_data set, if the number of reviews is equal to the maximum number of reviews for that application name *and* I have not already added this application to the cleaned dataset, then I will append the entire row to google_play_clean. Then I'll use my explore_data function to ensure that the process worked as expected.

In [ ]:
google_play_clean = []
already_added = []
for row in google_play_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        google_play_clean.append(row)
        already_added.append(name)
explore_data(google_play_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


So, I've removed the duplicate app entries in the Google Play data set. Thankfully, I don't need to do the same for the App Store data because there are no duplicates. If you don't trust me, you can check out the data set yourself which I linked above. However, there is still more to be done in the data cleaning process! Next, I will remove data from apps that are non-english, as this would be irrelevant to the product Harvestly is building soon. I will show an example of these apps below:

In [ ]:
print(google_play_clean[7940][0])

لعبة تقدر تربح DZ


In order to remove apps with non-English names, I will iterate through the data sets seeking out characters with ASCII values greater than 127, which will indicate that an application has non-english characters. This isn't a perfect system as there may be English applications with a character or two that have ASCII values greater than 127. So to get around this edge case, I will create a function that returns false only if the passed string has three or more characters that have ASCII values greater than 127.

In [ ]:
def isEnglish(str):
    count = 0
    for char in str:
        if (ord(char) > 127):
            count += 1
            if (count >= 3): 
                return False
    return True

Now that I have this function, I'll use it to iterate through the App Store and Google Play data sets and extract the English apps. I'll start by initializing two new empty lists (iOS and android) and append an app if it has an English title. Afterwards, I'll quickly use the explore_data function to ensure that my new lists iOS and android have been populated as expected.

In [ ]:
iOS = []
android = []
for row in app_store_data[1:]:
    name = row[2]
    if (isEnglish(name)):
        iOS.append(row)

for row in google_play_clean:
    name = row[0]
    if (isEnglish(name)):
        android.append(row)
        
explore_data(iOS, 0, 3, True)
explore_data(android, 0 , 3, True)

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


Number of rows: 6155
Number of columns: 17
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Te

Great! Next, I want to delete data that pertains to all applications that are not free to download. The reason for this is because the Harvestly mobile application will be completely free. The main source of revenue will not be from individuals purchasing the app or from in-app advertisements, but from enabling more people to use Harvestly's products and services through their mobile phone. In order to accomplish this, I will once again loop through both the iOS data and the android data and append all free apps to two new lists.

In [ ]:
iOS_free = []
android_free = []

for row in iOS:
    price = float(row[5])
    if price == 0.0:
        iOS_free.append(row)

for row in android:
    category = row[6]
    if category.lower() == 'free':
        android_free.append(row)
        
explore_data(iOS_free, 0, 3, True)
explore_data(android_free, 0, 3, True)

['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shopping', '128512000', 'USD', '0', '262241', '649', '4', '4.5', '5.10.0', '12+', 'Shopping', '37', '5', '9', '1']


Number of rows: 3203
Number of columns: 17
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', 

Great! Now I've got two lists, iOS_free and android_free, that both contain data from the App Store and from the Google Play store. Importantly, I can rest assured that this data is free from expensive apps, non-English apps, apps with inaccurate data, and apps with duplicate entries. This means that I'm just about ready to start analyzing the data. In order to create a relevant and compelling app profile for Harvestly to model, I'm first going to create some frequency tables and find out the most popular genres on the market.

In [ ]:
def freq_table(dataset, index):
    dictionary = {}
    for row in dataset:
        data = row[index]
        if data in dictionary:
            dictionary[data] += 1
        else:
            dictionary[data] = 1
    # return values as percentages
    total_number_of_apps = len(dataset)
    for i in dictionary:
        dictionary[i] /= total_number_of_apps
        dictionary[i] *= 100
    return dictionary

## sort percentages in descending order and display frequency table
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', round(entry[0], 3))
        
display_table(iOS_free, 12)
print('\n')
display_table(android_free, 1)
print('\n')
display_table(android_free, 9)
print('\n')

Games : 58.258
Entertainment : 7.836
Photo & Video : 4.995
Education : 3.684
Social Networking : 3.309
Shopping : 2.591
Utilities : 2.466
Sports : 2.154
Music : 2.061
Health & Fitness : 2.029
Productivity : 1.748
Lifestyle : 1.561
News : 1.342
Travel : 1.249
Finance : 1.093
Weather : 0.874
Food & Drink : 0.812
Reference : 0.531
Business : 0.531
Book : 0.375
Navigation : 0.187
Medical : 0.187
Catalogs : 0.125


FAMILY : 18.933
GAME : 9.698
TOOLS : 8.455
BUSINESS : 4.6
PRODUCTIVITY : 3.9
LIFESTYLE : 3.888
FINANCE : 3.707
MEDICAL : 3.538
SPORTS : 3.391
PERSONALIZATION : 3.323
COMMUNICATION : 3.233
HEALTH_AND_FITNESS : 3.086
PHOTOGRAPHY : 2.95
NEWS_AND_MAGAZINES : 2.803
SOCIAL : 2.668
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.249
BOOKS_AND_REFERENCE : 2.136
DATING : 1.865
VIDEO_PLAYERS : 1.797
MAPS_AND_NAVIGATION : 1.39
FOOD_AND_DRINK : 1.243
EDUCATION : 1.164
ENTERTAINMENT : 0.961
LIBRARIES_AND_DEMO : 0.938
AUTO_AND_VEHICLES : 0.927
HOUSE_AND_HOME : 0.803
WEATHER : 0.791
EVENTS : 0.712
PARENTI

By analyzing the frequency tables above, we can see which apps are most popular in each app store. For example, it becomes immediately clear that most of the apps on the App Store are designed for entertainment and fun, whereas apps on Google Play have a broader distribution between entertainment and practicality. However, these frequency tables do not provide enough for determining how to develop Harvestly's mobile application. It is important to note that just because there is a larger number of apps in a specific genre, that does not directly correlate to popularity, success, or profitability. So, to determine which apps are the most popular, I will calculate the average number of installs for each genre. I can do this easily in the Google Play data set, which has a column designated to installations, but will have to find a workaround for the App Store. To analyze the App Store data set, I will instead use the total number of user ratings column. I'll calculate the average number of user ratings in the App Store in the next code block by (1) isolating the apps of each genre, (2) summing up the user ratings for the apps of that genre, and (3) dividing the sum by the number of apps in that genre.

In [ ]:
genre_frequency_table = freq_table(iOS_free, 12)
for genre in genre_frequency_table:
    total = 0
    len_genre = 0
    for row in iOS_free:
        genre_app = row[12]
        if genre_app == genre:
            total += float(row[6])
            len_genre += 1
    avg_num_user_ratings = total/len_genre
    print(genre, round(avg_num_user_ratings, 3))

Productivity 21028.411
Weather 52279.893
Shopping 27230.735
Reference 79350.471
Finance 32367.029
Music 57326.53
Utilities 19156.494
Travel 28243.8
Social Networking 71548.349
Sports 23008.899
Health & Fitness 23298.015
Games 22886.367
Food & Drink 33333.923
News 21248.023
Book 46384.917
Photo & Video 28441.544
Entertainment 14195.359
Business 7491.118
Lifestyle 16815.48
Education 7003.983
Navigation 86090.333
Medical 612.0
Catalogs 4004.0


Great, now that that's done, I can do the same for the Google Play data set.

In [ ]:
category_frequency_table = freq_table(android_free, 1)
for genre in category_frequency_table:
    total = 0
    len_category = 0
    for row in android_free:
        category_app = row[1]
        if category_app == genre:
            installs = row[5]
            installs = installs.replace('+','')
            installs = installs.replace(',','')
            total += float(installs)
            len_category += 1
    avg_num_of_installs = total/len_category
    print(genre, round(avg_num_of_installs, 3))

ART_AND_DESIGN 1986335.088
AUTO_AND_VEHICLES 647317.817
BEAUTY 513151.887
BOOKS_AND_REFERENCE 8814199.788
BUSINESS 1712290.147
COMICS 832613.889
COMMUNICATION 38590581.087
DATING 854028.83
EDUCATION 1833495.146
ENTERTAINMENT 11640705.882
EVENTS 253542.222
FINANCE 1387692.476
FOOD_AND_DRINK 1924897.736
HEALTH_AND_FITNESS 4188821.985
HOUSE_AND_HOME 1360598.042
LIBRARIES_AND_DEMO 638503.735
LIFESTYLE 1446158.224
GAME 15544014.51
FAMILY 3697848.173
MEDICAL 120550.62
SOCIAL 23253652.127
SHOPPING 7036877.312
PHOTOGRAPHY 17840110.402
SPORTS 3650602.277
TRAVEL_AND_LOCAL 13984077.71
TOOLS 10830251.971
PERSONALIZATION 5201482.612
PRODUCTIVITY 16787331.345
PARENTING 542603.621
WEATHER 5145550.286
VIDEO_PLAYERS 24727872.453
NEWS_AND_MAGAZINES 9549178.468
MAPS_AND_NAVIGATION 4049274.634


It seems from the data above that the 'Shopping' category is much more popular in the Google Play store than in the App Store. This shouldn't surprise us, as it was clear that the Google Play store had a larger focus on practicality than the App Store did. This shows that our developer team at Harvestly will be better off creating a native android application before working on the iOS version, as the android platform is more likely to be more successful for Harvestly's needs.